In [ ]:
import mysql.connector
import pandas as pd
import os

# Connection details
conn = mysql.connector.connect(
    host='localhost',
    user='root',                    # Your MySQL username
    password=os.environ['MYSQL_PASSWORD'],                # Your MySQL password
    database='tawos'                # Name of the imported database
)

In [3]:
tables = pd.read_sql("SHOW TABLES;", conn)
print(tables)

     Tables_in_tawos
0   affected_version
1         change_log
2            comment
3          component
4        fix_version
5              issue
6    issue_component
7         issue_link
8            project
9         repository
10            sprint
11              user
12           version


C:\Users\Marc\AppData\Local\Temp\ipykernel_3464\2834186916.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql("SHOW TABLES;", conn)


In [4]:
# --- Load the full issue table ---
issue_df = pd.read_sql("SELECT * FROM issue;", conn)

# --- BASIC OVERVIEW ---
print("✅ Shape:", issue_df.shape)
print("\n✅ Column names:\n", issue_df.columns.tolist())

print("\n✅ Null counts (Top 10):\n", issue_df.isnull().sum().sort_values(ascending=False).head(10))
print("\n✅ Unique 'Type' values:\n", issue_df['Type'].value_counts(dropna=False))
print("\n✅ Unique 'Status' values:\n", issue_df['Status'].value_counts(dropna=False))
print("\n✅ Unique 'Priority' values:\n", issue_df['Priority'].value_counts(dropna=False))

# --- Preview sample data for important columns ---
preview_cols = [
    'Title', 'Description_Text', 'Type', 'Priority', 'Status',
    'Resolution', 'Story_Point', 'Timespent', 'Total_Effort_Minutes',
    'Sprint_ID', 'Assignee_ID', 'Project_ID'
]

print("\n📋 Sample rows:")
display(issue_df[preview_cols].sample(5, random_state=42))

# --- FILTER: Only user stories ---
story_df = issue_df[issue_df['Type'].str.contains('Story', na=False)].copy()
print(f"\n📚 Total user stories: {len(story_df)}")

# --- CLEANING ---
cols_to_keep = preview_cols + ['ID']
story_df = story_df[cols_to_keep]
story_df = story_df.dropna(subset=['Title', 'Description_Text'])

# --- EXAMPLES from Key Columns ---
print("\n📝 Sample Titles:")
print(story_df['Title'].dropna().sample(3, random_state=1).tolist())

print("\n🧾 Sample Descriptions:")
print(story_df['Description_Text'].dropna().sample(2, random_state=2).tolist())

print("\n⚙️ Sample Effort Estimates:")
print(story_df[['Story_Point', 'Timespent', 'Total_Effort_Minutes']].dropna().sample(3, random_state=3))

print("\n📦 Sample Sprint + Project Mapping:")
print(story_df[['Sprint_ID', 'Project_ID']].dropna().sample(3, random_state=4))

# --- FORMAT FOR LLM TRAINING PREVIEW ---
story_df['formatted'] = story_df.apply(lambda row: 
    f"Sprint_ID: {row['Sprint_ID']}\nUser Story: {row['Title']}\nDescription:\n{row['Description_Text']}", axis=1)

print("\n🧠 LLM Training Example:")
print(story_df['formatted'].sample(1).values[0])

C:\Users\Marc\AppData\Local\Temp\ipykernel_3464\3546174574.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  issue_df = pd.read_sql("SELECT * FROM issue;", conn)


✅ Shape: (458232, 30)

✅ Column names:
 ['ID', 'Jira_ID', 'Issue_Key', 'URL', 'Title', 'Description', 'Description_Text', 'Description_Code', 'Type', 'Priority', 'Status', 'Resolution', 'Creation_Date', 'Estimation_Date', 'Resolution_Date', 'Last_Updated', 'Story_Point', 'Timespent', 'In_Progress_Minutes', 'Total_Effort_Minutes', 'Resolution_Time_Minutes', 'Title_Changed_After_Estimation', 'Description_Changed_After_Estimation', 'Story_Point_Changed_After_Estimation', 'Pull_Request_URL', 'Creator_ID', 'Reporter_ID', 'Assignee_ID', 'Project_ID', 'Sprint_ID']

✅ Null counts (Top 10):
 Pull_Request_URL    453727
Timespent           447989
Sprint_ID           414757
Story_Point         392805
Estimation_Date     392805
Assignee_ID         196401
Resolution_Date     105600
Resolution           77498
Reporter_ID           2496
Creator_ID            1591
dtype: int64

✅ Unique 'Type' values:
 Type
Bug                              215570
Suggestion                        96370
Improvement     

,Title,Description_Text,Type,Priority,Status,Resolution,Story_Point,Timespent,Total_Effort_Minutes,Sprint_ID,Assignee_ID,Project_ID
51190,"""User & committer list rationalisation""",""""""" /users - all users in FECRU /users/REPO ...",Suggestion,,Closed,Fixed,NaN,240.0,19099.0,NaN,13553.0,17
232453,"""Write charge for LSST Science Platform Review ""","""""""Write the charge for the internal LSST Scie...",Story,,Done,Done,NaN,NaN,0.0,NaN,145743.0,28
216005,"""Only allow SCC plugins with build flag""",,Sub-task,Highest,Closed,Done,NaN,NaN,60.0,NaN,144463.0,24
407180,"""Extension Installer - Uninstalling does not a...","""""""Install 7.3.1 as FQDN site w/remote SQL db ...",Bug,High,Closed,Won't Fix,NaN,NaN,0.0,NaN,NaN,42
199924,"""We don't index user details""","""""""Right now, we don't index user details when...",Bug,Medium,Closed,Fixed,NaN,NaN,0.0,NaN,NaN,22



📚 Total user stories: 31394

📝 Sample Titles:
['"Membership services should support running in a cluster for HA"', '"Test if IPMI can replace KVM"', '"Create a new module launch configuration to support multiple platforms module packaging"']

🧾 Sample Descriptions:
['"""Create a simple set of code that simulates loading scripts and can be used to quickly try different solutions to speed up script loading.    This continues the work started in DM-21175 to speed up script loading.  """', '"""Example02 plays a prominent role in our documentation and tutorials. However, its name is historical and meaningless, and with the cleanup of fabric/examples looks very odd on its own.    Renaming it to something like ABstore would better convey what it does."""']

⚙️ Sample Effort Estimates:
        Story_Point  Timespent  Total_Effort_Minutes
243601          4.0      360.0               13161.0
223731          1.0      480.0              116515.0
246557          2.0      720.0                   0.

In [ ]:
# Step 1: Load both tables
issues = pd.read_sql("SELECT * FROM issue;", conn)
sprints = pd.read_sql("SELECT * FROM sprint;", conn)

# Keep only relevant columns
issues = issues[['ID', 'Title', 'Description_Text', 'Type', 'Sprint_ID', 'Project_ID', 'Story_Point', 'Timespent']]
sprints = sprints[['ID', 'Name', 'Start_Date', 'End_Date', 'Project_ID']]

# Filter for user stories
user_stories = issues[issues['Type'].str.contains('Story', na=False)].copy()

# Group by sprint and count
stories_by_sprint = user_stories.groupby('Sprint_ID').agg({
    'ID': 'count',
    'Story_Point': 'sum',
    'Title': lambda x: list(x)
}).rename(columns={
    'ID': 'num_stories',
    'Story_Point': 'total_story_points',
    'Title': 'titles'
}).reset_index()

# Merge with sprint metadata
stories_by_sprint = stories_by_sprint.merge(
    sprints,
    left_on='Sprint_ID',
    right_on='ID',
    how='left',
    suffixes=('', '_sprint')
)

# Show top rows
print("📊 Grouped user stories by sprint:\n")
print(stories_by_sprint.head())

# Optional: View all story titles in a specific sprint
sprint_example = stories_by_sprint['Sprint_ID'].dropna().iloc[0]
print(f"\n📝 Titles in Sprint {sprint_example}:\n")
print(user_stories[user_stories['Sprint_ID'] == sprint_example][['Title', 'Description_Text']].head())


C:\Users\Marc\AppData\Local\Temp\ipykernel_3464\817123874.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  issues = pd.read_sql("SELECT * FROM issue;", conn)
C:\Users\Marc\AppData\Local\Temp\ipykernel_3464\817123874.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sprints = pd.read_sql("SELECT * FROM sprint;", conn)


📊 Grouped user stories by sprint:

   Sprint_ID       Name           Start_Date             End_Date  \
0        4.0  Sprint 63  2015-11-30 16:32:01  2015-12-11 17:28:00   
1        5.0  Sprint 62  2015-11-16 16:10:26  2015-11-27 17:06:00   
2        6.0  Sprint 61  2015-11-02 17:41:25  2015-11-14 00:37:00   
3        7.0  Sprint 60  2015-10-19 17:04:56  2015-10-31 00:00:00   
4        8.0  Sprint 59  2015-10-05 16:54:15  2015-10-17 00:44:00   

   num_stories  total_story_points  
0            3                10.0  
1            6                12.0  
2            9                27.0  
3           25                77.0  
4           13                30.0  

📝 Titles in Sprint 4.0:

                                  Title  \
53    "Move k8s SPI to a separate repo"   
54  "Upgrade XD Ambari release to 1.3 "   
96  "Move Mesos SPI to a separate repo"   

                                     Description_Text  
53  """As a developer, I'd like to move k8s SPI to...  
54  """As a devel

KeyboardInterrupt: 